# Perguntas

Qual filme você recomendaria para uma pessoa que você não conhece?

Quais são os principais fatores que estão relacionados com alta expectativa de faturamento de um filme?
 
Quais insights podem ser tirados com a coluna Overview? É possível inferir o gênero do filme a partir dessa coluna?


In [1]:
# Importando as bibliotecas

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [2]:
# Lendo os dados

df = pd.read_csv('https://raw.githubusercontent.com/pat-weber/lighthouse/main/lighthouse_project/data/desafio_indicium_imdb.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


In [3]:
# Padronizando as colunas

df.columns = df.columns.str.lower()
df['released_year'] = df['released_year'].replace('PG', 0)

# Transformando os dados dos anos em numéricos
df['released_year'] = df['released_year'].astype(int)
df['runtime'] = df['runtime'].str.replace('min', '').astype(int)
df['gross'] = df['gross'].str.replace(',', '').astype(float)

# Preenchendo valores ausentes
df['gross'] = df['gross'].fillna(df['gross'].median())
df['meta_score'] = df['meta_score'].fillna(df['meta_score'].median())


### Qual filme você recomendaria para uma pessoa que você não conhece?

Para uma pessoa desconhecida, uma possível abordagem é guiar pela popularidade do filme, que se expressa tanto pelas avaliações, quanto pelo faturamento. 

In [4]:
# Selecionando filmes que têm uma nota IMDB maior que 8, avaliação maior que 80 e tiveram maior faturamento
recommended_movies = df[(df['imdb_rating'] >= 8.0) & (df['meta_score'] >= 80) & (df['gross'] >= df['gross'].quantile(0.75))]
recommended_movies.sort_values(by='gross', ascending=False)

# Imprimindo os 5 com maior faturamento
recommended_movies.head(5)

,unnamed: 0,series_title,released_year,certificate,runtime,genre,imdb_rating,overview,meta_score,director,star1,star2,star3,star4,no_of_votes,gross
0,1,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
1,2,The Dark Knight,2008,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0
4,5,The Lord of the Rings: The Return of the King,2003,U,201,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,377845905.0
5,6,Pulp Fiction,1994,A,154,"Crime, Drama",8.9,"The lives of two mob hitmen, a boxer, a gangst...",94.0,Quentin Tarantino,John Travolta,Uma Thurman,Samuel L. Jackson,Bruce Willis,1826188,107928762.0
6,7,Schindler's List,1993,A,195,"Biography, Drama, History",8.9,"In German-occupied Poland during World War II,...",94.0,Steven Spielberg,Liam Neeson,Ralph Fiennes,Ben Kingsley,Caroline Goodall,1213505,96898818.0


Também podemos selecionar as recomendações pelas avaliações. Já que um alto faturamento pode ser em parte fruto de uma boa campanha de lançamento. E as avaliações conferem uma nota real do sentimento do público em relação ao filme.

In [5]:
# Selecionando os 5 filmes com maiores avaliações
recommended_movies = df[(df['imdb_rating'] >= 8.0) & (df['meta_score'] >= 80) & (df['gross'] >= df['gross'].quantile(0.75))]
recommended_movies_score = recommended_movies.sort_values(by='meta_score', ascending=False)
recommended_movies_score.head(5)

,unnamed: 0,series_title,released_year,certificate,runtime,genre,imdb_rating,overview,meta_score,director,star1,star2,star3,star4,no_of_votes,gross
0,1,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
313,314,Gone with the Wind,1939,U,238,"Drama, History, Romance",8.1,A manipulative woman and a roguish man conduct...,97.0,Victor Fleming,George Cukor,Sam Wood,Clark Gable,Vivien Leigh,290074,198676459.0
366,367,Ratatouille,2007,U,111,"Animation, Adventure, Comedy",8.0,A rat who can cook makes an unusual alliance w...,96.0,Brad Bird,Jan Pinkava,Brad Garrett,Lou Romano,Patton Oswalt,641645,206445654.0
65,66,WALL·E,2008,U,98,"Animation, Adventure, Family",8.4,"In the distant future, a small waste-collectin...",95.0,Andrew Stanton,Ben Burtt,Elissa Knight,Jeff Garlin,Fred Willard,999790,223808164.0
400,401,Beauty and the Beast,1991,G,84,"Animation, Family, Fantasy",8.0,A prince cursed to spend his days as a hideous...,95.0,Gary Trousdale,Kirk Wise,Paige O'Hara,Robby Benson,Jesse Corti,417178,218967620.0


O único título em comum nas duas listas é O Poderoso Chefão, esse título seria uma recomendação lógica para uma pessoa adulta desconhecida.

### Quais são os principais fatores que estão relacionados com alta expectativa de faturamento de um filme? 

Na análise exploratória de dados vimos que o gênero de aventura tem o maior faturamento. Vamos testar um modelo de análise preditiva nesta mesma base de dados.

In [6]:
# Aplicando Label Encoder
le = LabelEncoder()

# Ajustando e transformando cada coluna de gênero
for col in ['certificate', 'genre', 'director', 'star1']:
    df[col] = le.fit_transform(df[col].astype(str))

# Selecionando as características e objetivo
X = df[['certificate', 'released_year', 'runtime', 'genre', 'meta_score', 'no_of_votes', 'imdb_rating']]
y = df['gross']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Treinando o modelo de floresta aleatória
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Avaliando o modelo
y_pred_rf = rf_model.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf)

print(f'R² Score (Floresta Aleatória): {r2_rf}')

R² Score (Floresta Aleatória): 0.6782497643459944


In [8]:
# Treinando o modelo de árvore de decisão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

# Avaliando o modelo
y_pred_tree = tree_model.predict(X_test)
r2_tree = r2_score(y_test, y_pred_tree)

print(f'R² Score (Árvore de Decisão): {r2_tree}')

R² Score (Árvore de Decisão): 0.4885549788848802


In [9]:
# Treinando o modelo de regressão linear
rl_model = LinearRegression()
rl_model.fit(X_train, y_train)

# Avaliando o modelo
y_pred_rl = rl_model.predict(X_test)
r2_rl = r2_score(y_test, y_pred_rl)

print(f'R² Score (Regressão Linear): {r2_rl}')

R² Score (Regressão Linear): 0.4887886674843316


O modelo com a melhor performance é a Floresta aleatória, que consegue captar boa parte das variáveis relativas ao faturamento de um filme, mas ainda não com uma previsão apurada suficiente que possa indicar as diretrizes da produção de um filme de alto faturamento. 

### Quais insights podem ser tirados com a coluna Overview? É possível inferir o gênero do filme a partir dessa coluna?

Vamos testar o modelo de Floresta aleatória, que mostrou o melhor desempenho com essa base de dados na questão anterior. Para isso vamos precisar transformar as sinopses em números que o modelo consiga analisar.

In [10]:
# Criando um objeto TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Aplicando o TfidfVectorizer na coluna 'overview'
overview_tfidf = tfidf_vectorizer.fit_transform(df['overview'])

# Convertendo a saída para um DataFrame
overview_tfidf_df = pd.DataFrame(overview_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenando com o DataFrame original
df_overview = pd.concat([df, overview_tfidf_df], axis=1)

# Visualizando o DataFrame resultante
df_overview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Columns: 5440 entries, unnamed: 0 to édith
dtypes: float64(5427), int64(8), object(5)
memory usage: 41.5+ MB


In [11]:
# Declarando características e objetivos
X_overview = df_overview.drop(['unnamed: 0', 'series_title', 'genre', 'overview', 'star2', 'star3', 'star4'], axis=1)
y_overview = df_overview['genre']

# Dividir os dados em treino e teste
X_over_train, X_over_test, y_over_train, y_over_test = train_test_split(X_overview, y_overview, test_size=0.2, random_state=42)

In [12]:
# Treinando o modelo de floresta aleatória
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_over_train, y_over_train)

# Avaliando o modelo
y_over_pred = rf_model.predict(X_over_test)
r2_rf = r2_score(y_over_test, y_over_pred)
mse_rf = mean_squared_error(y_over_test, y_over_pred)
rmse_rf = np.sqrt(mse_rf)

print(f'R² Score (Floresta Aleatória): {r2_rf}')

R² Score (Floresta Aleatória): 0.00021073122754833307


Esse valor de R2 é muito baixo para ser considerado modelo viável, mesmo realizando ajustes. Esse é possívelmente uma questão que demanda um modelo de maior complexidade de análise. 

Vamos testar uma outra abordagem: criar um ciclo que busque as palavras descritivas dos gêneros nas sinopses na coluna overview. 

In [13]:
# Lendo os dados
df_genre = pd.read_csv('https://raw.githubusercontent.com/pat-weber/lighthouse/main/lighthouse_project/data/desafio_indicium_imdb.csv')

# Padronizando as colunas
df_genre.columns = df_genre.columns.str.lower()

# Lista de palavras-chave e suas respectivas categorias
keywords_to_category = {
    'Crime': 'Crime',
    'Drama': 'Drama',
    'Action': 'Action',
    'Adventure': 'Adventure',
    'Biography': 'Biography',
    'History': 'History',
    'Romance': 'Romance',
    'Fantasy': 'Fantasy',
    'War': 'War',
    'Thriller': 'Thriller',
    'Animation': 'Animation',
    'Comedy': 'Comedy',
    'Sci-Fi': 'Sci-Fi',
    'Family': 'Family',
    'Western': 'Western',
    'Mystery': 'Mystery',
    'Horror': 'Horror',
    'Sport': 'Sport',
    'Music': 'Music'
}

# Função para classificar o gênero com base nas palavras-chave
def classify_genre(overview):
    for keyword, category in keywords_to_category.items():
        if keyword in overview:
            return category
    return 'Other'  

# Aplicando a função para classificar o gênero
df_genre['genre_classified'] = df_genre['overview'].apply(classify_genre)

# Visualizando quantas classificações foram realizadas
genre_counts = df_genre['genre_classified'].value_counts()
print(genre_counts)

genre_classified
Other      947
War         48
Western      2
Romance      1
Sport        1
Name: count, dtype: int64


Também não é uma abordagem interessante, apenas 52 títulos foram classificados.

In [14]:
filtered_comparison = df_genre[df_genre['genre_classified'] != 'Other'][['genre', 'genre_classified']]

print(filtered_comparison.shape)


(52, 2)


In [15]:
# Filtrando as linhas onde genre_classified não é 'Other'
filtered_comparison = df_genre[df_genre['genre_classified'] != 'Other'][['genre', 'genre_classified']]

# Verificando se o gênero classificado está presente na coluna genre
filtered_comparison['is_in_genre'] = filtered_comparison.apply(lambda row: row['genre_classified'] in row['genre'], axis=1)

# Verificando apenas as linhas onde o gênero classificado está na coluna genre
comparison_true = filtered_comparison[filtered_comparison['is_in_genre']]
print(comparison_true.shape)
comparison_true


(19, 3)


,genre,genre_classified,is_in_genre
45,"Animation, Drama, War",War,True
92,"Adventure, Drama, War",War,True
103,"Drama, War",War,True
104,"Drama, Thriller, War",War,True
114,Western,Western,True
235,"Action, Drama, War",War,True
277,"Drama, War",War,True
407,"Drama, War",War,True
429,"Drama, Romance, War",War,True
444,"Comedy, Drama, War",War,True


É possível que para o gênero de guerra essa abordagem consiga inferir o gênero, ainda assim, de forma restrita, pois não capta sub-gêneros. 